In [1]:
%load_ext autoreload
%autoreload 2

# BTM用に､10000分割したテキストをre-clusteringするための分配処理

In [2]:
#loaderの設定などの練習
import json
from loaders.CommonCrawlDataset import CommonCrawlDataset

In [3]:
#10000分割されたjsonlファイルのパスを取得

import glob
import random

jsonl_path_list=glob.glob("../data/dedup_categorized/*/*.jsonl")
random.shuffle(jsonl_path_list)
cid_to_path_list={}
for jsonl_path in jsonl_path_list:
    cid=jsonl_path.split("/")[-2]
    if cid not in cid_to_path_list:
        cid_to_path_list[cid]=[]
    cid_to_path_list[cid].append(jsonl_path)

In [4]:
from tqdm import tqdm
#代表的な*件を取得
max_texts=10
cid_to_texts={}
for cid, path_list in tqdm(cid_to_path_list.items()):
    texts=[]
    for path in path_list:
        cnt=0
        with open(path) as f:
            for i, line in enumerate(f):
                t=json.loads(line)["text"]
                if len(t)<10:
                    continue
                texts.append(t)
                cnt+=1
                if cnt>max_texts:
                    break
    cid_to_texts[cid]=texts

  0%|          | 0/3815 [00:00<?, ?it/s]

100%|██████████| 3815/3815 [00:00<00:00, 4733.66it/s]


In [5]:
import sys
import numpy as np
sys.path.append("../01web_codes")
from gensim.models.fasttext import load_facebook_model
from tqdm import tqdm
from src.classify.Text2Vec import Text2Vec

t2v = Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))

In [6]:
#各カテゴリのテキストをベクトル化
cid_to_vecs={}
check_length=100
for cid, texts in tqdm(cid_to_texts.items()):
    vecs = [t2v.text2vec(i[:check_length]) for i in texts]
    vecs=np.array(vecs)
    vecs=np.mean(vecs, axis=0)
    cid_to_vecs[cid]=vecs



100%|██████████| 3815/3815 [00:05<00:00, 755.41it/s] 


In [7]:
vec_array=list(cid_to_vecs.values())
vec_array=[i for i in vec_array if i.shape==(300,)]
vecs.shape

(300,)

In [8]:
n_clusters=5
from sklearn.cluster import MiniBatchKMeans,KMeans
vec_array=np.array(vec_array).astype(np.double)
# k-meansクラスタリング
print("clustering...")
kmeans = KMeans(n_clusters=n_clusters, random_state=1).fit(vec_array)

clustering...


In [9]:
cid_to_label={}
for cid, vec in cid_to_vecs.items():
    try:
        label=kmeans.predict([vec])[0]
        cid_to_label[cid]=label
    except Exception as e:
        print(cid,e)

In [10]:
label_to_path_list={}
for cid, label in cid_to_label.items():
    if str(label) not in label_to_path_list:
        label_to_path_list[str(label)]=[]
    path=f"../data/dedup_categorized/{cid}/*.jsonl"
    label_to_path_list[str(label)].append(path)


In [11]:
import json
with open("../data/clustered_path.json", "w") as f:
    json.dump(label_to_path_list, f, indent=4)

In [12]:
#行数を数える
import os
import subprocess
import json
from tqdm import tqdm
from loaders.CommonCrawlDataset import CommonCrawlDataset

with open("../data/clustered_path.json") as f:
    label_to_path_list = json.load(f)



label_to_article_count = {}

for label, path_list in tqdm(label_to_path_list.items()):
    loader = CommonCrawlDataset(label_to_path_list[label])
    loader=iter(loader)
    while True:
        try:
            next(loader)
        except:
            records=loader.count
            label_to_article_count[label] = records
            break

label_to_article_count

100%|██████████| 5/5 [00:00<00:00,  5.69it/s]


{'1': 7826, '2': 6053, '3': 12775, '4': 5617, '0': 4325}

In [13]:

with open("../data/clustered_n.json", "w") as f:
    json.dump(label_to_article_count, f, indent=4)

In [14]:
n_clusters=5
from loaders.CommonCrawlDataset import CommonCrawlDataset
for i in range(n_clusters):
    print("--------\ncluster: ",i)
    cc1_loader=CommonCrawlDataset(label_to_path_list[str(i)])
    cc1_iter=iter(cc1_loader)
    for i in range(10):
        print(next(cc1_iter))

--------
cluster:  0
{'text': 'ブラックブラウン了承しました了承しました軒先渡し・お客様組立品となります\n北海道/東北/九州/沖縄/離島については、別途送料お見積りさせていただきます。\n3月21日20時から!\nPeppaホスピタル・ビルディングセット,フレンズMr.Potato&Spud Spiderヘッドハスブロスパイダーマン,【送料無料】おけいこクマさんアタッチメント・ベアーカレイドスコープペンダント&中央の宝石は5mm×5mmの存在感のあるペリドット。彫りの装飾はなくシンプルなフォルム。緑のベネチアンガラスにはいくつものお花が描かれています。'}
{'text': '〈1面〉軽減税率は導入確実与党が議論再開「生鮮」か「精米」か事務処理なお曲折\n自民、公明の与党両党は5月22日、2017年4月から導入予定の消費税10%時の軽減税率議論を本格的にスタートざせた。これを受けて、公明党は5月27日午前8時に税調総会を開いた。また、同日の午後3時から与党は2回目の会合を開き、3回目は検討期間を置き6月9日に開く。財務省案が示され、これによると、軽減品目について「すべての飲食料品」から「精米のみ」の8案(原案)の中から、1酒類を除く飲食良品2生鮮食品3精米の3案に絞って、課題を整理した。この案を見る限り、17年4月の消費税10%導入時に2%をめどに軽減税率が17年度中に導入されるは確実な情勢になった。\n〈2面〉女性行員が人材交流女性起業家と意見交換も千葉・東邦・第四・北国・中国・伊予銀\nシステム共同化を検討する「TSUBASA(翼)プロジェクト」の参加行である千葉銀行、東邦銀行、第四銀行、北国銀行、中国銀行、伊予銀行は5月21日、22日に「輝く女性のためのシンガポールミッション」を実施した。6行の女性行員の交流は翼プロジェクトでは初めてで、15人(各行2人～4人)が参加した。参加者の国際感覚を高めて人的ネットワークを形成するのが狙いだ。ミッションでは東南アジアにおける金融の最先端であるシンガポールでの企業視察や現地で活躍する日本の女性起業家との意見交換を通して業務効率化、女性キャリアアップ、リテール金融などを考察した。エンディングでは各班に分けてプレゼンテーションも行った。\n燕市長鈴木氏「産業そのものを観光に」天草市長中村氏「補

In [15]:
#モデル保存
import pickle
with open("../data/second_kmeans_model.pkl", "wb") as f:
    pickle.dump(kmeans, f)